In [ ]:
import numpy as np
import csv

# Преобразование изображения в черно-белое
def convert_to_grayscale(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        grayscale_image = np.mean(image, axis=2).astype(np.uint8)
    else:
        grayscale_image = image
    return grayscale_image

# Подавление шума (медианный фильтр)
def median_filter(image, filter_size):
    height, width = image.shape
    filtered_image = np.zeros((height, width), dtype=np.uint8)
    radius = filter_size // 2

    for i in range(radius, height - radius):
        for j in range(radius, width - radius):
            neighborhood = image[i-radius:i+radius+1, j-radius:j+radius+1]
            filtered_image[i, j] = np.median(neighborhood)

    return filtered_image

# Повышение резкости границ (оператор Собеля)
def sobel_edges(image):
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])

    def convolve(image, kernel):
        height, width = image.shape
        k_height, k_width = kernel.shape
        pad_h, pad_w = k_height // 2, k_width // 2
        padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
        convolved_image = np.zeros_like(image)

        for i in range(height):
            for j in range(width):
                region = padded_image[i:i+k_height, j:j+k_width]
                convolved_image[i, j] = np.sum(region * kernel)

        return convolved_image

    gradient_x = convolve(image, sobel_x)
    gradient_y = convolve(image, sobel_y)
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)

    return gradient_magnitude.astype(np.uint8)

# Удаление теней
def remove_shadows(image):
    threshold_value = 255
    shadow_mask = image < threshold_value
    image[shadow_mask] = 255
    return image

# Выравнивание изображения
def equalize_image(image):
    normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))
    equalized_image = (normalized_image * 255).astype(np.uint8)
    return equalized_image

# Удаление углов
def remove_corners(image):
    height, width = image.shape
    for i in range(height):
        for j in range(width):
            if (i < height * 0.1 or i > height * 0.9) and (j < width * 0.1 or j > width * 0.9):
                image[i, j] = 255
    return image

# Коррекция перспективы
def correct_perspective(image, src_points, dst_points):
    src_matrix = np.array([[src_points[0][0], src_points[1][0], src_points[2][0], src_points[3][0]],
                           [src_points[0][1], src_points[1][1], src_points[2][1], src_points[3][1]],
                           [1, 1, 1, 1]])

    dst_matrix = np.array([[dst_points[0][0], dst_points[1][0], dst_points[2][0], dst_points[3][0]],
                           [dst_points[0][1], dst_points[1][1], dst_points[2][1], dst_points[3][1]],
                           [1, 1, 1, 1]])

    transform_matrix = np.dot(dst_matrix, np.linalg.inv(src_matrix))

    def apply_transform(x, y, matrix):
        vector = np.array([x, y, 1])
        transformed_vector = np.dot(matrix, vector)
        transformed_vector /= transformed_vector[2]
        return int(transformed_vector[0]), int(transformed_vector[1])

    height, width = image.shape
    corrected_image = np.zeros_like(image)

    for i in range(height):
        for j in range(width):
            new_x, new_y = apply_transform(j, i, transform_matrix)
            if 0 <= new_x < width and 0 <= new_y < height:
                corrected_image[new_y, new_x] = image[i, j]

    return corrected_image

# Преобразование в бинарное изображение
def binary_threshold(image, threshold):
    binary_image = np.where(image < threshold, 0, 255)
    return binary_image

# Обнаружение и удаление сетки ЭКГ
def remove_grid(image):
    horizontal_kernel = np.array([[-1, -1, -1],
                                  [2,  2,  2],
                                  [-1, -1, -1]])

    vertical_kernel = np.array([[-1, 2, -1],
                                [-1, 2, -1],
                                [-1, 2, -1]])

    def apply_kernel(image, kernel):
        height, width = image.shape
        k_height, k_width = kernel.shape
        pad_h, pad_w = k_height // 2, k_width // 2
        padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
        result_image = np.zeros_like(image)

        for i in range(height):
            for j in range(width):
                region = padded_image[i:i+k_height, j:j+k_width]
                result_image[i, j] = np.sum(region * kernel)

        return result_image

    horizontal_lines = apply_kernel(image, horizontal_kernel)
    vertical_lines = apply_kernel(image, vertical_kernel)
    grid_mask = (horizontal_lines > 128) | (vertical_lines > 128)
    image[grid_mask] = 255  # Удаляем сетку, закрашивая её белым цветом

    return image

# Обнаружение сигнала ЭКГ
def detect_ecg_signal(image):
    mean_intensity = np.mean(image)
    peak_threshold = mean_intensity * 0.9
    peak_mask = image > peak_threshold
    return peak_mask

# Извлечение сигнала ЭКГ
def extract_ecg_signal(binary_image):
    horizontal_lines = np.sum(binary_image, axis=1)
    line_threshold = np.max(horizontal_lines) * 0.2
    line_indices = np.where(horizontal_lines > line_threshold)[0]
    ecg_signal = np.zeros(binary_image.shape[0], dtype=np.uint8)
    ecg_signal[line_indices] = 255
    return ecg_signal

# Объединение сигнала и его извлечение в CSV
def extract_and_save_ecg_signal(binary_image, output_csv_path):
    horizontal_lines = np.sum(binary_image, axis=1)
    line_threshold = np.max(horizontal_lines) * 0.6
    line_indices = np.where(horizontal_lines > line_threshold)[0]
    ecg_signal = []
    for index in line_indices:
        ecg_signal.append(binary_image[index, :])
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(ecg_signal)
    print(f"ECG signal has been saved to {output_csv_path}")